In [13]:
import numpy as np
import moviepy as mpy

from moviepy.editor import *

# from moviepy import *

# import librosa

# from cvtools import ipynb_show_matrix
# from datasets import CS4065_Dataset

# %matplotlib inline

print mpy.__version__

0.2.2.11


## Video Retrieval

This final assignment is about soccer match summarization.
This notebook shows video summarization based on peak volume of commentator voice.

In [14]:
# clipfull = VideoFileClip("soccer.mp4")
clipfull = VideoFileClip("soccer-subclip.mp4")
offset = 20 #the real kick of / start of the match
livescore_offset = -60 + offset; # livescore use ceiling for the event's timestamp

include_goals = True;
include_fouls = True;

# Get subclip for second half
# clipsecond = VideoFileClip("soccer.mp4").subclip("01:17:00")

In [15]:
# Show video and show properties upon click
# test.preview(fps=29)

# Show frame on exact second and show properties upon click
# test.show(1, interactive = True)

# Show the clip, time & memory consuming
# Could be exceed max time duration
# ipython_display(clipsecond)

## Video Processing

Video processing to get the audio signal

In [16]:
# Function to create sound array for i
cut = lambda i: clipfull.audio.subclip(i,i+1).to_soundarray(fps=44100)

# Function to get volume from sound array
volume = lambda array: np.sqrt(((1.0*array)**2).mean())

volumes = [volume(cut(i)) for i in range(0,int(clipfull.audio.duration))]

In [17]:
print volumes

[0.044085378537103155, 0.042569819296773344, 0.060001041992513916, 0.064911589964606733, 0.049844174400126964, 0.055157363573401008, 0.062514981505746595, 0.052259551579659363, 0.056554584489468554, 0.05303561909044336, 0.061698121761109927, 0.043843432222805741, 0.06885733082893658, 0.063647734095984038, 0.060128021873656526, 0.05493260300721467, 0.045598059649948376, 0.059680581255106238, 0.080276960267180245, 0.081676909575510617, 0.075178539504873229, 0.072425594288300199, 0.065793020106745287, 0.062531705472931298, 0.058085131933484677, 0.067125702474517893, 0.063048333906944615, 0.058657229676995742, 0.057882247775463296, 0.045605840709458993, 0.054630987295980059, 0.049716630365565367, 0.069381654291874387, 0.05039281701856306, 0.068480669786989912, 0.078154714569790146, 0.055575064995282537, 0.062981711056387904, 0.081078266569097793, 0.066064968802755017, 0.061042103330166832, 0.056104660337376017, 0.053240297195632176, 0.061139286799268049, 0.062156791985163044, 0.05489433682

In [26]:
# Get volumes for each 10s
# averaged_volumes = np.array([sum(volumes[i:i+10])/10
#                              for i in range(len(volumes)-10)])

vol = np.array(volumes)

In [27]:
# help(np.diff)

In [30]:
# increases = np.diff(averaged_volumes)[:-1]>=0
# decreases = np.diff(averaged_volumes)[1:]<=0
# peaks_times = (increases * decreases).nonzero()[0]
# peaks_vols = averaged_volumes[peaks_times]
# peaks_times = peaks_times[peaks_vols>np.percentile(peaks_vols,90)]

increases = np.diff(vol)[:-1]>=0
decreases = np.diff(vol)[1:]<=0
peaks_times = (increases * decreases).nonzero()[0]
peaks_vols = vol[peaks_times]
peaks_times = peaks_times[peaks_vols>np.percentile(peaks_vols,90)]

In [38]:
print "Audio: peaks times"
print peaks_times

# Livescore times
print "Livescore: Goals times"
goals_times = np.array([360, 840, 2700])
# Correct the livescore offser
goals_times = goals_times + livescore_offset 
print goals_times

print "Livescore: Fouls times"
fouls_times = np.array([1200, 1500, 2460])
# Correct the livescore offser
fouls_times = fouls_times + livescore_offset 
print fouls_times


Audio: peaks times
[  18  119  130  135  139  145  150  162  167  169  174  259  288  290  294
  296  302  310  321  356  411  427  534  542  546  548  550  553  556  653
  656  660  709  732  772  776  780  783  785  788  791  793  796  953  972
 1083 1387 1390 1456 1524 1531 1542 1690 1738 1740 1888 2018 2072 2081 2085
 2121 2124 2129 2133 2138 2141 2223 2225 2227 2241 2243 2250 2252 2460 2516
 2518 2526 2531 2533 2536 2539 2542 2705 2708 2736 2739 2826 2900 2905 2929]
Livescore: Goals times
[ 320  800 2660]
Livescore: Fouls times
[1160 1460 2420]


In [39]:
# final_times=[peaks_times[0]]
# for t in peaks_times:
#     if (t - final_times[-1]) < 60:
#         if averaged_volumes[t] > averaged_volumes[final_times[-1]]:
#             final_times[-1] = t
#     else:
#         final_times.append(t)
        

final_times=[peaks_times[0]]
for t in peaks_times:
    if (t - final_times[-1]) < 60:
        if vol[t] > vol[final_times[-1]]:
            final_times[-1] = t
    else:
        final_times.append(t)
        
print "Audio: final grouped"
print final_times

if include_goals:
    print "Merge Audio with Livescore - Goals"
    final_times = np.sort(np.append(final_times,goals_times))
    print final_times
    
if include_fouls:
    print "Merge Audio with Livescore - Fouls"
    final_times = np.sort(np.append(final_times,fouls_times))
    print final_times


Audio: final grouped
[18, 167, 310, 427, 546, 653, 785, 972, 1083, 1390, 1456, 1542, 1738, 1888, 2124, 2223, 2518, 2736, 2826, 2900]
Merge Audio with Livescore - Goals
[  18  167  310  320  427  546  653  785  800  972 1083 1390 1456 1542 1738
 1888 2124 2223 2518 2660 2736 2826 2900]
Merge Audio with Livescore - Fouls
[  18  167  310  320  427  546  653  785  800  972 1083 1160 1390 1456 1460
 1542 1738 1888 2124 2223 2420 2518 2660 2736 2826 2900]


In [ ]:
# TODO: make it dynamic using audio approach
foi_window = 15;

final = concatenate([clipfull.subclip(max(t-foi_window,0),min(t+foi_window, clipfull.duration)) for t in peaks_times])
final.to_videofile("soccer-summary-livescore-and-audio.mp4") # low quality is the default

[MoviePy] >>>> Building video soccer-summary-livescore-and-audio.mp4
[MoviePy] Writing audio in soccer-summary-livescore-and-audioTEMP_MPY_wvf_snd.mp3


  2%|▏         | 1348/59337 [00:05<03:42, 260.19it/s]